# Compute single unit metrics for each session

1 - either use all annotated & uploaded ephys sessions as input or provide a list of session_ids

2 - choose whether to overwrite exsiting files

3 - set a savepath and filename for the output - one .pkl file per session

4 - compute metrics

(see description of metrics below)

In [7]:
import pandas as pd
import npc_lims
# from npc_sessions import DynamicRoutingSession
from dynamic_routing_analysis import spike_utils
from dynamic_routing_analysis import data_utils
# import dynamic_routing_analysis as dra
import os
import pickle
import upath

%load_ext autoreload
%autoreload 2
# %matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#get all uploaded & annotated ephys sessions
# ephys_sessions=tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, is_annotated=True,))

In [2]:
#alternatively, provide a list of session ids:
session_id_list=['742903_2024-10-22']
session_list=[]
for ss in session_id_list:
    session_list.append(npc_lims.get_session_info(ss))
ephys_sessions=tuple(session_list)

In [ ]:
except_dict={}
overwrite_files=False
# save_path = dra.SINGLE_UNIT_METRICS_PATH
save_path= upath.UPath(r'D:\test_single_unit_metrics')

for session_info in ephys_sessions[:]:
    
    # try:
        trials=[]
        units=[]
        ##skip if file already exists
        
        if overwrite_files==False and (save_path / f"{session_info.id}_stim_context_modulation.pkl").exists():
            print('session',session_info.id,'already processed; skipping')
            continue
        lick_save_path=os.path.join(save_path,'lick_modulation')
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        # try:
        # session=DynamicRoutingSession(session_info.id)
        session_id=session_info.id
        trials=data_utils.load_trials_or_units(session_id, 'trials', version='0.0.272')
        units=data_utils.load_trials_or_units(session_id, 'units', version='0.0.272')

        # except Exception as e:
        #     print('failed to load trials or units, skipping session')
        #     except_dict[session_info.id]=e
        #     continue

        lick_mod=spike_utils.compute_lick_modulation(trials, units, session_info, save_path=None, test=True)

        stim_mod=spike_utils.compute_stim_context_modulation(trials, units, session_info, save_path=None, test=True)

        merged_df=stim_mod.merge(lick_mod, on=['unit_id','session_id','project'], how='left')

        merged_df.to_parquet(save_path / f"{session_info.id}_stim_context_lick_modulation_refactor.parquet")

        # print(session_info.id,'done')

    # except Exception as e:
    #     print(session_info.id,'failed')
    #     except_dict[session_info.id]=e


742903_2024-10-22_0 cached trials loaded
742903_2024-10-22_0 cached units loaded


KeyError: 'cr_vs_fa_early_roc_auc'

In [ ]:
except_dict

In [17]:
stim_mod=spike_utils.compute_stim_context_modulation(trials, units, session_info, save_path=None, test=True)

C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:633: RuntimeWarning: invalid value encountered in divide
  )/(stim_frs_by_trial.mean(dim='trials',skipna=True)+stim_baseline_frs_by_trial.mean(dim='trials',skipna=True))
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:652: RuntimeWarning: invalid value encountered in divide
  stim_late_mod_sign=np.sign(stim_late_frs_by_trial.mean(dim='trials',skipna=True)-stim_baseline_frs_by_trial.mean(dim='trials',skipna=True))
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:667: RuntimeWarning: invalid value encountered in divide
  
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:670: RuntimeWarning: invalid value encountered in divide
  
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:633: RuntimeWarning: divid

In [10]:
stim_mod

,index,activity_drift,amplitude,amplitude_cutoff,amplitude_cv_median,amplitude_cv_range,amplitude_median,ccf_ap,ccf_dv,ccf_ml,...,catch_evoked_context_modulation_p_value,catch_evoked_context_modulation_zscore,catch_evoked_context_modulation_sign,catch_evoked_context_modulation_raw,catch_context_modulation_index,catch_evoked_context_modulation_index,catch_stimulus_modulation_roc_auc,catch_stimulus_late_modulation_roc_auc,catch_context_modulation_roc_auc,catch_evoked_context_modulation_roc_auc
0,1011,0.040825,295.738655,0.000024,0.119623,0.091381,236.340000,4450.0,4400.0,4950.0,...,0.751524,0.253670,1.0,1.355556,0.431818,5.545455,0.497296,0.581666,0.622222,0.527778
1,1073,0.039527,88.367762,0.002602,0.325891,0.232566,60.839996,4450.0,4400.0,4950.0,...,1.000000,NaN,0.0,0.000000,NaN,NaN,0.500000,0.500000,0.500000,0.500000
2,1053,0.079220,98.602840,0.000758,0.207254,0.089202,72.540000,4400.0,4325.0,4975.0,...,1.000000,NaN,0.0,0.000000,NaN,NaN,0.500000,0.511628,0.500000,0.500000
3,33,0.086092,69.306116,0.002313,0.336214,0.109470,44.460000,3850.0,2500.0,4850.0,...,0.257899,-inf,-1.0,-0.555556,-1.000000,-1.000000,0.511628,0.500000,0.472222,0.472222
4,18,0.100674,167.244541,0.003975,NaN,NaN,126.359990,3850.0,2500.0,4850.0,...,1.000000,0.000000,0.0,0.000000,-1.000000,NaN,0.500000,0.500000,0.472222,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4441,820,0.086122,204.300610,0.000053,0.163228,0.162902,152.099990,3250.0,3800.0,5350.0,...,0.402223,0.422872,1.0,1.355556,0.180328,5.545455,0.511628,0.501352,0.524444,0.558889
4442,819,0.138864,143.006752,0.000011,0.214832,0.167319,91.259995,3250.0,3800.0,5350.0,...,0.392045,-0.639336,-1.0,-8.977778,-0.040799,-1.364865,0.555435,0.533802,0.490000,0.424444
4443,818,0.077887,119.653568,0.000084,0.297627,0.158574,79.560000,3250.0,3800.0,5350.0,...,0.248214,0.495432,1.0,2.222222,0.090909,-1.000000,0.453488,0.529746,0.516667,0.593333
4444,817,0.068722,482.929321,0.000007,0.176435,0.149985,332.280000,3250.0,3750.0,5350.0,...,0.744588,0.176190,1.0,2.066667,0.017199,-1.631579,0.471877,0.545430,0.525556,0.530000


In [ ]:
# df=pd.read_parquet(r"D:\test_single_unit_metrics\742903_2024-10-22_0_stim_context_modulation.parquet")
# df
for col in merged_df.columns:
    print(col, merged_df[col].dtype)

In [ ]:
merged_df.drop(columns=['spike_times','waveform_sd','waveform_mean','obs_intervals'], inplace=True, errors='ignore')

In [ ]:
test_units=data_utils.load_trials_or_units(session_id, 'units', version='0.0.272')


In [ ]:
test_units['electrode_group_name'].iloc[0]

In [ ]:
type(merged_df['electrode_group_name'].iloc[0])

In [ ]:
import polars as pl
merged_pl=pl.DataFrame(merged_df)
merged_pl.write
# stim_mod.to_parquet(r"D:\test_single_unit_metrics\test_0.parquet")

## single unit metrics descriptions


### baseline context modulation

comparisons between all visual context vs. auditory context trials

time window = [-0.1, 0] relative to stimulus onset

**'baseline_context_modulation_index'** - (vis-aud)/(vis+aud)

**'baseline_context_modulation_p_value'** - scipy.stats.mannwhitneyu(vis,aud)

**'baseline_context_modulation_zscore'** - (vis-aud)/std(all trials)

**'baseline_context_modulation_sign'** - np.sign(np.mean(vis-aud))

**'baseline_context_roc_auc'** - sklearn.metrics.roc_auc_score(context_label,baseline_frs)

**'linear_shift_baseline_context_true_value'** - mean (vis-aud) baseline fr difference, properly aligned

**'linear_shift_baseline_context_null_median'** - median (vis-aud) across all shifts

**'linear_shift_baseline_context_null_mean'** - mean (vis-aud) across all shifts

'**linear_shift_baseline_context_null_std**' - std of (vis-aud) across all shifts

'**linear_shift_baseline_context_p_value_higher**' - the fraction of shift values that the true value is higher than

'**linear_shift_baseline_context_p_value_lower**' - the fraction of shift values that the true value is lower than



### stimulus modulation

comparisons between baseline and stimulus firing rates per trial

baseline time window = [-0.1,0] sec relative to stimulus onset

stimulus time window = [0,0.1]

late stimulus time window = [0.1,0.2]

replace "stim" with specific stimulus name: vis1, vis2, sound1, sound2, or catch

'**stim_stimulus_modulation_index**' - (stim-baseline)/(stim+baseline)

'**stim_stimulus_modulation_zscore**' - (stim-baseline)/std(baseline)

'**stim_stimulus_modulation_p_value**' - scipy.stats.wilcoxon(stim,baseline)

'**stim_stimulus_modulation_sign**' - np.sign(np.mean(stim-baseline))

'**stim_stimulus_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(stim_vs_baseline_label,frs)

'**stim_stimulus_late_modulation_index**' - same as above but for later time window

'**stim_stimulus_late_modulation_zscore**' 

'**stim_stimulus_late_modulation_p_value**'

'**stim_stimulus_late_modulation_sign**'

'**stim_stimulus_late_modulation_roc_auc**'

'**stim_stim_latency**' - latency of the maximum response in the [0,0.3] time window - in units of binsize (default=0.025sec)



### context modulation of stimulus response

comparison between stimulus responses in same vs. other contexts

(i.e. for vis1, same context = vis, other context = aud)

time window = [0,0.1] relative to stimulus onset

'**stim_context_modulation_index**' - (stim_same_context-stim_other_context)/(stim_same_context+stim_other_context)

'**stim_context_modulation_zscore**' - (stim_same_context-stim_other_context)/(stim_same_context+stim_other_context)

'**stim_context_modulation_sign**' - np.sign(np.mean(stim_same_context-stim_other_context))

'**stim_context_modulation_p_value**' - scipy.stats.mannwhitneyu(stim_same_context,stim_other_context)

'**stim_context_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(context_label,stim_frs)

'**stim_evoked_context_modulation_index**' - same as above but for evoked fr - subtract the [-0.1,0] baseline per trial

'**stim_evoked_context_modulation_zscore**'

'**stim_evoked_context_modulation_sign**'

'**stim_evoked_context_modulation_p_value**'



### lick modulation

comparisons between correct reject (no_lick) and false alarm (lick) trials

time window: [0.2,0.5]

baseline: [-0.5,-0.2]

'**lick_modulation_index**' - (lick-no_lick)/(lick+no_lick)

'**lick_modulation_zscore**' - (lick-no_lick)/std(baseline)

'**lick_modulation_p_value**' - st.mannwhitneyu(lick,no_lick)

'**lick_modulation_sign**' - np.sign(np.mean(lick-no_lick))

'**lick_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(lick, no_lick)



### stimulus selectivity

comparisons between firing rate responses to different stimuli

time window: [0,0.1]

'**vis_discrim_roc_auc**' - vis1 vs. vis2

'**aud_discrim_roc_auc**' - sound1 vs. sound2

'**target_discrim_roc_auc**' - vis1 vs. sound1

'**nontarget_discrim_roc_auc**' - vis2 vs. sound2

'**vis_vs_aud**' - all vis vs. all sound


### trial response type selectivity

comparisons between response types, in different time windows

all stimuli included - maybe should be

time windows [0,0.1] [0.1,0.2] [0.2,0.3]

'cr_vs_fa_early_roc_auc'

'hit_vs_cr_early_roc_auc'

'hit_vs_fa_early_roc_auc'

'cr_vs_fa_mid_roc_auc'

'hit_vs_cr_mid_roc_auc'

'hit_vs_fa_mid_roc_auc'

'cr_vs_fa_late_roc_auc'

'hit_vs_cr_late_roc_auc'

'hit_vs_fa_late_roc_auc'
